In [33]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [34]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix
        if len(self.sig_scattering_matrix) > 1:
            for i in range(0, len(self.sig_scattering_matrix)):
                self.sig_s.append(sum(self.sig_scattering_matrix[i]))
        else:
            self.sig_s =  self.sig_scattering_matrix
                   
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_number_of_groups(self):
        
        return self.number_groups
        

In [35]:
energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [36]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group

    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [71]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [72]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        if self.xs[0].number_groups == 1:
            self.delta_xs = self.xs[0].sig_t
            
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [73]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [74]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [75]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [76]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [77]:
class Multigroup_estimator:
    def __init__(self):
        self.groups_before = []
        self.weights_before = []
        self.groups_after = []
        self.weights_after = []
        
    def get_flux_ration_n_m(self, first, secound):
        
        number_before = 0.
        number_after = 0.
        for i in range(0, len(self.groups_before)):
            
            if self.groups_before[i] == first:
                number_before += self.weights_before[i]           
                if self.groups_after[i] == secound:
                    number_after += self.weights_after[i]
        
        flux_ratio = number_after / number_before
    
        return flux_ratio   

In [78]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [79]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [80]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [81]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)     

In [82]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    

In [83]:
def process_real_collision(particle, free_path, c_s, flux_group_estimator):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':
        process_scattering(particle, c_s)

        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [84]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [85]:
def process_one_particle_history(particle, universe, estimators, flux_estimator):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    while not particle.is_terminated():
        
        energy_group_before = particle.get_energy_group()
        weight_before = particle.get_weight()
        

        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        flux_estimator.groups_before.append(particle.get_energy_group())
        flux_estimator.weights_before.append(particle.get_weight())
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions 
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs, flux_estimator)
            
    
    return particle, sum_collisions

In [86]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [87]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])  

In [88]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [89]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [90]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [91]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [92]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [93]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [94]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [95]:
class Flux_group_estimator:
    def __init__(self, group):
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if particle.get_energy_group() == self.group:
            self.collision_sum[-1]  += particle.get_weight()    

In [96]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    flux_group_estimators = []
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    number_of_groups = universe.xs[0].get_number_of_groups()
    for i in range(0, number_interations):
        print("i == " + str(i))
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        flux_group_estimator = Multigroup_estimator()
        for j in range(0, batch_size):
            particle = initial_sources[j]           
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, 
                                                                            estimators, flux_group_estimator)
            batch_particles.append(terminate_particle)
            
            
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        flux_group_estimators.append(flux_group_estimator)
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s)              

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)
     
    return k_effective, k_effective_exp, k_effective_std, estimators, flux_group_estimators

In [97]:
cs_fission_pu_239 = [0.081600]

cs_capture_pu_239 = [0.019584]

cs_scattering_pu_239 = [0.225216]

cs_total_pu_239 = [0.32640]

cs_production_neutrons_pu_239 = [3.24]

cs_xi_table_pu_239 = [1.]


pu_239 = Sig(6, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [98]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0, -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000., -1000, 1000, 1000., 1000 )
slab.set_zero_point(0., 0., 0.)

In [99]:
cells = [slab]
materials = [pu_239]
U235_Reactor = Universe(cells, materials)

In [100]:
estimators = []

In [101]:
test_number_of_particles = 20
test_number_interations = 1000
test_number_inactive = 200

In [102]:
k_effective,k_effective_exp, k_effective_std, estimators_result, flux_group_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)

 num1  7
i == 0


IndexError: list index out of range

In [105]:
first_group_flux = []
secound_group_flux = []
third_group_flux = []

group4_flux = []
group5_flux = []
group6_flux = []

cs_total_pu_239 = [0.240, 0.975, 3.10, 3.10, 0.975, 0.240]

for i in range(200, len(estimators_result[0].collision_sum) - 1):
    first_group_flux.append(estimators_result[0].collision_sum[i]/(cs_total_pu_239[0]))

for i in range(200, len(estimators_result[1].collision_sum) - 1):
    secound_group_flux.append(estimators_result[1].collision_sum[i]/(cs_total_pu_239[1]))

for i in range(200, len(estimators_result[2].collision_sum) - 1):
    third_group_flux.append(estimators_result[2].collision_sum[i]/(cs_total_pu_239[2]))
    
    
for i in range(200, len(estimators_result[3].collision_sum) - 1):
    group4_flux.append(estimators_result[3].collision_sum[i]/(cs_total_pu_239[3]))

for i in range(200, len(estimators_result[4].collision_sum) - 1):
    group5_flux.append(estimators_result[4].collision_sum[i]/(cs_total_pu_239[4]))

for i in range(200, len(estimators_result[5].collision_sum) - 1):
    group6_flux.append(estimators_result[5].collision_sum[i]/(cs_total_pu_239[5]))
    

average_flux_difference_1 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_1.append(secound_group_flux[i] / first_group_flux[i])
    
average_flux_difference_2 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_2.append(third_group_flux[i] / secound_group_flux[i])
    
average_flux_difference_3 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_3.append(third_group_flux[i] / first_group_flux[i])
    
average_flux_difference_4 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_4.append(group5_flux[i] /group6_flux[i])
    
average_flux_difference_5 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_5.append(group4_flux[i] / group6_flux[i])
    
average_flux_difference_6 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_6.append(group4_flux[i] / group5_flux[i])

    
std_flux_1 = get_std(average_flux_difference_1)
flux_difference_1 = sum(average_flux_difference_1)/len(average_flux_difference_1)

std_flux_2 = get_std(average_flux_difference_2)
flux_difference_2 = sum(average_flux_difference_2)/len(average_flux_difference_2)

std_flux_3 = get_std(average_flux_difference_3)
flux_difference_3 = sum(average_flux_difference_3)/len(average_flux_difference_3)


std_flux_4 = get_std(average_flux_difference_4)
flux_difference_4 = sum(average_flux_difference_4)/len(average_flux_difference_4)

std_flux_5 = get_std(average_flux_difference_5)
flux_difference_5 = sum(average_flux_difference_5)/len(average_flux_difference_5)

std_flux_6 = get_std(average_flux_difference_6)
flux_difference_6 = sum(average_flux_difference_6)/len(average_flux_difference_6)

In [106]:
print(average_flux_difference_4)

[0.4806188650287858, 0.48812343306450323, 0.49996146648144096, 0.5065626918576679, 0.46102093054242443, 0.49771267548585507, 0.45798543168701805, 0.44721475016998796, 0.5015279240478563, 0.46758011116545745, 0.466965732493996, 0.4897029528987381, 0.47236588140949987, 0.4777514127890825, 0.4885351868262832, 0.4813762301435226, 0.4932796869101505, 0.49815470927904687, 0.4803423859812391, 0.46118746611472905, 0.44742830392635763, 0.4653567299621209, 0.5015994243835472, 0.4489359986829114, 0.483955414990258, 0.45246007669414207, 0.5028732706381485, 0.4687916745596713, 0.47489034007913766, 0.49916948679656187, 0.49815769907423163, 0.43058505943952025, 0.5191623789934703, 0.47354938896363385, 0.4863976434828582, 0.4698046109037066, 0.47255693721801956, 0.5103976570624131, 0.46574182944611375, 0.4742470895934762, 0.49736881122467375, 0.47864904005933856, 0.4608740735511713, 0.4678200607093473, 0.4768664909251361, 0.43196134435520855, 0.46112554151464524, 0.44352837669279854, 0.507386204149331

In [107]:
difference = (flux_difference_1 - 0.480) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_1 * 100000))

group 1 to group 2 flux difference flux == 13.235125160165895with std   33.557784709515424


In [108]:
difference = (flux_difference_2 - 0.3125) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_2 * 100000))

group 1 to group 2 flux difference flux == 30.11346300023643with std   20.77279915202315


In [109]:
difference = (flux_difference_3 - 0.150) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_3 * 100000))

group 1 to group 2 flux difference flux == 7.488527369031717with std   9.548357845478476


In [111]:
print(flux_difference_4)

0.48411736274880834


In [112]:
difference = (flux_difference_4 - 0.480) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_4 * 100000))

group 1 to group 2 flux difference flux == 411.7362748808362with std   262.2910441144333


In [113]:
difference = (flux_difference_5 - 0.150) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_5 * 100000))

group 1 to group 2 flux difference flux == 128.50772191167758with std   75.77647721333953


In [114]:
difference = (flux_difference_6 - 0.3125) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_6 * 100000))

group 1 to group 2 flux difference flux == 37.466906054567374with std   20.314041384375276


In [115]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [116]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)

In [117]:
def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()

In [118]:
file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)
